In [1]:
import torch
from collections import OrderedDict
import pickle
import os
import glob
from PIL import Image
from torchvision import transforms

c:\ProgramFiles\anaconda3\envs\MMSA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a7_path = "D:/Desktop/code/bishe/pre-data/video/affecnet7_epoch6_acc0.6569.pth"
a8_path = "D:/Desktop/code/bishe/pre-data/video/affecnet8_epoch5_acc0.6209.pth"

In [3]:
from torch import nn
from torchvision.models import resnet18 

resnet = resnet18()
features_model = nn.Sequential(*list(resnet.children())[:-2])

missing_keys, unexpected_keys = features_model.load_state_dict(torch.load(a7_path, map_location='cpu'), strict=False)
# missing_keys, unexpected_keys = features.load_state_dict(weight, strict=False)    


In [5]:
img_path = r"D:\Desktop\code\6th-ABAW\test_data\openface_img"

In [6]:

def totenser_normal(img):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    return transform(img)
    
img_dirs = glob.glob(os.path.join(img_path, '*'))
img_dirs.sort()
# print(len(img_dirs))
# img_dirs = img_dirs[1721:]
# img_dirs = img_dirs[11694:]
# print(len(img_dirs),img_dirs[0])

save_dir = r"D:\Desktop\code\6th-ABAW\test_data\resnet18"
if not os.path.exists(save_dir): os.makedirs(save_dir)

for img_dir in img_dirs:

    imgs = glob.glob(os.path.join(img_dir, '*'))
    imgs.sort(key=lambda x: int(os.path.basename(x)[:-4]))
    # print(imgs)
    if len(imgs) != 0:
        seg_imgs = [totenser_normal(Image.open(img).convert('RGB')) for img in imgs]
        # print(totenser_normal(seg_imgs[0]).shape)
        seg_imgs = torch.stack(seg_imgs) # (t, 3, 112, 112)
        # print(seg_imgs.shape)
        
        # model
        feature = features_model(seg_imgs)
        feature = torch.mean(feature, dim=(-2, -1))
        
        # pickle.dumps
        # print(img_dir[-4:])
        p = os.path.join(save_dir, img_dir[-5:])
        with open(f'{p}.pkl', 'wb') as f:
            pickle.dump(feature, f)

    else:
        p = os.path.join(save_dir, img_dir[-5:])
        # with open('kong_file.txt', 'a') as f1:
        #     f1.write(p + '\n')
        print(p)

        with open(f'{p}.pkl', 'wb') as f:
            feature = torch.randn(5, 512) 
            pickle.dump(feature, f)
   


D:\Desktop\code\6th-ABAW\test_data\resnet18\16626


In [10]:
import pickle

with open(r'D:\Desktop\code\6th-ABAW\dataset\openface_aus\11694.pkl', 'rb') as f:
    data = pickle.load(f)

# data = data.detach().numpy()
print(data)
# feature = data[:200]
# print(feature.shape)

# with open(r'D:\Desktop\code\6th-ABAW\dataset\openface_aus\11694.pkl', 'wb') as f:
#     pickle.dump(feature, f)

# data.shape


[[0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
